# Verification of Quantum Circuits

Compiling quantum algorithms results in different representations of the considered functionality, which significantly differ in their basis operations and structure but are still supposed to be functionally equivalent.
As described in the previous section, even individual compilation tasks can be highly complex.
Consequently, checking whether the original functionality is indeed maintained throughout all these different abstractions becomes increasingly relevant in order to guarantee a consistent and error-free compilation flow.
This is similar to the classical realm, where descriptions at various levels of abstraction also exist.
These descriptions are verified using design automation expertise---resulting in efficient methods for verification to ensure the correctness of the design across different levels of abstraction.
However, since quantum circuits additionally employ quantum-physical effects such as superposition and entanglement, these methods cannot be used out of the box in the quantum realm.
Accordingly, verification of quantum circuits must be approached from a different perspective.
At first glance, these characteristics of quantum computing make verification much harder as for classical circuits and systems.
In fact, equivalence checking of quantum circuits has been proven to be a computationally hard problem.

At the same time, quantum circuits possess certain characteristics that offer remarkable potential for efficient equivalence checking that is not available in classical computing.
More precisely, consider two quantum circuits $G=g_1,\dots,g_m$ and $G'=g'_1,\dots,g'_n$ whose equivalence shall be checked.
Due to the inherent reversibility of quantum operations, the inverse of a quantum circuit can easily be computed by taking the complex conjugate of every gate and reversing the sequence of the gates in the circuit, i.e., $G^{\prime -1}= (g'_n)^\dagger,\dots,(g'_1)^\dagger$.
If two circuits are equivalent, this allows for the conclusion that $G\cdot G^{\prime -1} = I$, where $I$ is the identity function.
Since the identity has the most compact representation for most data structures representing quantum functionality (e.g., linear with respect to the number of qubits in case of decision diagrams), the equivalence check can be simplified considerably.
Even complex circuits can be verified efficiently, if one manages to apply the gates of both circuits in a sequence that keeps the intermediate representation "close to the identity".
Within the MQT, several methods and strategies were proposed that utilize this characteristic of quantum computations.
Eventually, this led to solutions that can verify the results of whole quantum compilation flows (such as IBM's Qiskit) in negligible runtime---something we never even managed for classical circuits and systems.

The _MQT_ offers the quantum circuit equivalence checking tool QCEC which encompasses a comprehensive suite of efficient methods and automated tools for the verification of quantum circuits.
By this, an important step towards avoiding or substantially mitigating the emerge of a verification gap for quantum circuits is taken, i.e., a situation where the physical development of a technology substantially outperforms our ability to design suitable applications for it or to verify it.


In [ ]:
from mqt.bench import get_benchmark, BenchmarkLevel
from qiskit.transpiler import CouplingMap
from qiskit.visualization import plot_gate_map
from qiskit.providers.fake_provider import GenericBackendV2

# Create a 5-qubit device with linear connectivity
num_qubits = 5
coupling_map = CouplingMap.from_line(num_qubits=num_qubits)
backend = GenericBackendV2(num_qubits=num_qubits, coupling_map=coupling_map, basis_gates=["cx", "u"])

# Visualize the device
plot_gate_map(backend)

## Setup: Create a Device and Load a Circuit

Let's start with a 5-qubit device and a quantum circuit to verify:

## Compare Original and Compiled Circuits

Let's load a Quantum Fourier Transform (QFT) circuit and compile it:

In [ ]:
# Load a 5-qubit QFT circuit (platform-independent level)
circ = get_benchmark(benchmark="qft", level=BenchmarkLevel.ALG, circuit_size=5)
circ.decompose().draw(output='mpl', fold=-1)

In [ ]:
from qiskit import transpile

# Compile the circuit to the device
circ_mapped = transpile(circ, backend=backend)
circ_mapped.draw(output='mpl', fold=-1)

## Examine the Underlying Matrices

Both circuits should implement the same unitary transformation. Let's check their matrix representations:

In [ ]:
from qiskit.quantum_info import Operator

# Extract the unitary matrix from the original circuit
op_initial = Operator.from_circuit(circ.remove_final_measurements(inplace=False))
print("Original circuit unitary matrix:")
print(op_initial.to_matrix())

In [ ]:
# Extract the unitary matrix from the compiled circuit
op_mapped = Operator.from_circuit(circ_mapped.remove_final_measurements(inplace=False))
print("Compiled circuit unitary matrix:")
print(op_mapped.to_matrix())

## Verification Method 1: Using Qiskit

Qiskit can compare operators by constructing their full matrix representations:

In [ ]:
# Check equivalence using Qiskit's built-in method
result = op_initial.equiv(op_mapped)
print(f"Circuits are equivalent: {result}")

## Verification Method 2: Using MQT QCEC

MQT QCEC uses decision diagrams and exploits reversibility for efficient verification:

In [ ]:
from mqt.qcec import verify

# Verify equivalence using MQT QCEC
result = verify(circ, circ_mapped)
print(f"Circuits are equivalent: {result.considered_equivalent()}")
print(f"Equivalence class: {result.equivalence}")

## Performance Comparison: QCEC vs Qiskit

Let's benchmark both verification methods on QFT circuits of increasing size:

In [ ]:
import time

times_MQT_QCEC = []
times_qiskit = []
qubit_range = range(2, 12)

for i in qubit_range:
    print(f"Verifying {i}-qubit QFT circuit...")
    
    # Load and compile circuit
    circ = get_benchmark(benchmark="qft", level=BenchmarkLevel.ALG, circuit_size=i)
    backend = GenericBackendV2(num_qubits=i, coupling_map=CouplingMap.from_line(i), 
                               basis_gates=["cx", "rz", "sx", "x"])
    circ_mapped = transpile(circ, backend=backend)
    
    # Measure MQT QCEC verification time
    start_time = time.time()
    assert verify(circ, circ_mapped).considered_equivalent()
    times_MQT_QCEC.append(time.time() - start_time)
    print(f"  MQT QCEC: {times_MQT_QCEC[-1]:.6f}s")

    # Measure Qiskit verification time
    start_time = time.time()
    op_initial = Operator.from_circuit(circ.remove_final_measurements(inplace=False))
    op_mapped = Operator.from_circuit(circ_mapped.remove_final_measurements(inplace=False))
    assert op_initial.equiv(op_mapped)
    times_qiskit.append(time.time() - start_time)
    print(f"  Qiskit: {times_qiskit[-1]:.6f}s")
    
print("\nBenchmark complete!")

In [ ]:
import matplotlib.pyplot as plt

# Plot verification time comparison
plt.figure(figsize=(10, 6))
plt.xlabel("Circuit Size (qubits)")
plt.ylabel("Verification Time (s)")
plt.title("Verification Performance: MQT QCEC vs Qiskit")
plt.plot(qubit_range, times_MQT_QCEC, marker='o', label="MQT QCEC")
plt.plot(qubit_range, times_qiskit, marker='s', label="Qiskit (Matrix Comparison)")
plt.legend()
plt.grid(True, alpha=0.3)
plt.yscale('log')  # Log scale to better show the difference
plt.show()

# Print speedup factor
print(f"\nFor {max(qubit_range)}-qubit circuits:")
print(f"MQT QCEC is {times_qiskit[-1]/times_MQT_QCEC[-1]:.1f}x faster than Qiskit")

## Exercises for Further Exploration

Try extending this tutorial with:

1. **Different Algorithms**: Verify other circuits from MQT Bench (e.g., `grover`, `shor`, `qaoa`)
   - See: https://mqt.readthedocs.io/projects/bench/en/latest/Benchmark_selection.html

2. **Different Compilers**: Compare verification of circuits compiled with:
   - Qiskit's different optimization levels (0-3)
   - MQT QMAP vs Qiskit transpiler

3. **Advanced QCEC Features**: Explore different verification strategies
   - See: https://mqt.readthedocs.io/projects/qcec/en/latest/

4. **Error Injection**: Intentionally modify compiled circuits and verify QCEC catches non-equivalence